# Zurich Instruments LabOne Python API Example
## Control the HDAWG in grouped mode

Demonstrate how to connect to a Zurich Instruments HDAWG in grouped mode. The grouped mode allows to control multiple 
AWG outputs with a single sequencer program.

Requirements:
* LabOne Version >= 22.08
* Instruments:
    1 x HDAWG Instrument

---

In [ ]:
import textwrap
import numpy as np
import time
import json, jsonschema

import zhinst.core
import zhinst.utils

Set up the connection. The connection is always through a session to a
Data Server. The Data Server then connects to the device.

The LabOne Data Server needs to run within the network, either on localhost when
starting LabOne on your local computer or a remote server.

In [ ]:
device_id = "dev8293"  # Device serial number available on its rear panel.
interface = "1GbE"  # For Ethernet connection.
# interface = "USB" # For all instruments connected to the host computer via USB.

server_host = "localhost"
server_port = 8004
api_level = 6  # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)
# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

# Create a base configuration: disable all available outputs, awgs,...
daq.syncSetInt(f"/{device_id}/system/preset/load", 1)
zhinst.utils.wait_for_state_change(daq, f"/{device_id:s}/system/preset/busy", 0, timeout = 2.0)

### Basic configuration
In this example the device is configured to control groups of 4 outputs with a single sequencer program (channel grouping 1). It is also possible to control groups of 2 outputs (channel grouping 0) or 8 outputs (channel grouping 2).

After specifying the grouping mode, specify which group to use and set the output gains for each AWG core in the group. Then set the output range for each channel in the group and switch the channels on.

In [ ]:
grouping = 1        # Channel grouping 2x4
awg_group = 0       # AWG group
output_range = 1.0  # Output range [V]

awg_cores = awg_group * 2**grouping + np.arange(2**grouping)        # AWG cores
channels = awg_group * 2**(grouping+1) + np.arange(2**(grouping+1)) # Output channels

# Grouping mode
exp_setting = [
    [f"/{device_id:s}/system/awg/channelgrouping", grouping]          
]

# Per-core settings
for awg_core in awg_cores:
    exp_setting_core = [
        [f"/{device_id:s}/awgs/{awg_core:d}/outputs/0/gains/0", 1.0],         # Set the output gains matrix to identity
        [f"/{device_id:s}/awgs/{awg_core:d}/outputs/0/gains/1", 0.0],         
        [f"/{device_id:s}/awgs/{awg_core:d}/outputs/1/gains/0", 0.0],         
        [f"/{device_id:s}/awgs/{awg_core:d}/outputs/1/gains/1", 1.0],
        [f"/{device_id:s}/awgs/{awg_core:d}/outputs/0/modulation/mode", 0],   # Turn off modulation mode
        [f"/{device_id:s}/awgs/{awg_core:d}/outputs/1/modulation/mode", 0]
    ]
    exp_setting.extend(exp_setting_core)

# Per-channel settings
for channel in channels:
    exp_setting_ch = [
        [f"/{device_id:s}/sigouts/{channel:d}/range", output_range],    # Select the output range
        [f"/{device_id:s}/sigouts/{channel:d}/on", True]                # Turn on the outputs. Should be the last setting
    ]
    exp_setting.extend(exp_setting_ch)

# Upload settings to the device in a transaction
daq.set(exp_setting)

### AWG sequencer program
Define an AWG program as a string stored in the variable `awg_program`, equivalent to what would be entered in the Sequence Editor window in the graphical UI. Differently to a self-contained program, this example refers to a command table by the instruction `executeTableEntry`, and to placeholder waveforms `p1`, `p2`, `p3`, `p4` by the instruction `placeholder`. Both the command table and the waveform data for the placeholders need to be uploaded separately before this sequence program can be run.

After defining the sequencer program, this must be compiled before being uploaded. The function `compile_seqc` of `zhinst-utils`, which is the preferred tool for compiling AWG programs, does not support working with the grouped mode. For this reason, in this example the compilation must be done using the LabOne module `awgModule`.

In [ ]:
wfm_index = 0
wfm_length = 1024
awg_program = textwrap.dedent(
    f"""\
    // Define placeholder with 1024 samples:
    wave p1 = placeholder({wfm_length:d});
    wave p2 = placeholder({wfm_length:d});
    wave p3 = placeholder({wfm_length:d});
    wave p4 = placeholder({wfm_length:d});

    // Assign an index to the placeholder waveform
    assignWaveIndex(1,p1, 2,p2, {wfm_index:d});
    assignWaveIndex(3,p3, 4,p4, {wfm_index:d});

    while(true) {{
      executeTableEntry(0);
    }}
    """
)

Compile and upload the AWG program to the device using the AWG Module.

In [ ]:
# Create an instance of the AWG Module
awgModule = daq.awgModule()
awgModule.set("device", device_id)
awgModule.set("index", awg_group)
awgModule.execute()

# Transfer the AWG sequencer program. Compilation starts automatically.
awgModule.set("compiler/sourcestring", awg_program)

Check that the sequencer program was compiled and uploaded correctly. This is not mandatory, but only to ensure that the script can continue with the next steps.

In [ ]:
# Wait until compilation is done
timeout = 10  # seconds
start = time.time()
compiler_status = awgModule.getInt("compiler/status")
while compiler_status == -1:
    if time.time() - start >= timeout:
        raise TimeoutError("Program compilation timed out")
    time.sleep(0.01)
    compiler_status = awgModule.getInt("compiler/status")

compiler_status_string = awgModule.getString("compiler/statusstring")
if compiler_status == 0:
    print(
        "Compilation successful with no warnings, will upload the program to the instrument."
    )
if compiler_status == 1:
    raise RuntimeError(
        f"Error during sequencer compilation: {compiler_status_string:s}"
    )
if compiler_status == 2:
    print(f"Warning during sequencer compilation:  {compiler_status_string:s}")

# Wait until the sequence is correctly uploaded
start = time.time()
for awg_core in awg_cores:
    # Check the ready status for each core
    while daq.getInt(f"/{device_id:s}/awgs/{awg_core:d}/ready") == 0:
        # Timeout if all the cores doesn't report ready in time
        if time.time() - start >= timeout:
            raise TimeoutError(f"Sequence not uploaded within {timeout:.1f}s.")
        time.sleep(0.01)

print("Sequence successfully uploaded.")

### Command Table definition and upload

The waveforms are played by a command table, whose structure must conform to a defined schema. The schema can be read from the device. This example validates the command table against the schema before uploading it. This step is not mandatory since the device will validate the schema as well. However, it is helpful for debugging.

Read the schema from the device.

In [ ]:
schema_node = f"/{device_id:s}/awgs/0/commandtable/schema"
schema = json.loads(
    daq.get(schema_node, flat=True)[schema_node][0]["vector"]
)
print(f"The device is using the commandtable schema version {schema['version']}")

Define two command tables and validate them against the schema.

In [ ]:
cts = [
# First command table
{   
    "header": {
        "version": "1.1.0",
    },
    "table": [
        {
            "index": 0,
            "waveform": {"index": wfm_index},
            "amplitude0": {"value": 1.0},
            "amplitude1": {"value": 1.0},
        }
    ],
},
# Second command table
{
    "header": {
        "version": "1.1.0",
    },
    "table": [
        {
            "index": 0,
            "waveform": {"index": wfm_index},
            "amplitude0": {"value": 0.5},
            "amplitude1": {"value": -1.0},
        }
    ],
}
]

for ct in cts:
    jsonschema.validate(
        instance=ct,
        schema=schema,
        cls=jsonschema.Draft7Validator,
    )

Upload the two command tables to the two AWG cores and check if the upload ends successfully.

In [ ]:
# Upload the command tables
ct_set = []
for ct, awg_c in zip(cts, awg_cores):
    ct_set.append([f"/{device_id:s}/awgs/{awg_c:d}/commandtable/data", json.dumps(ct,separators=(",", ":"))])

daq.set(ct_set)


# Wait until the command table is correctly uploaded
timeout = 10.0
start = time.time()
for awg_core in awg_cores:

    # Check the ready status for each core
    while (True):
        status = daq.getInt(f"/{device_id:s}/awgs/{awg_core:d}/commandtable/status")

        if status & 0b1:
            # Upload successful, move on the next core
            break

        if status & 0b1000:
            # Error in command table
            raise RuntimeError(f"The upload of command table on core {awg_core:d} failed.")

        if time.time() - start >= timeout:
            # Timeout if all the cores doesn't report ready in time
            raise TimeoutError(f"Commandtable not uploaded within {timeout:.1f}s.")
        
        time.sleep(0.01)

print("Command tables upload successful")

### Waveform upload

Replace the placeholder waveform with a drag pulse (I quadrature is a gaussian and Q quadrature is the derivative of I). The waveform data is uploaded to the index `wfm_index`, which must be the same specified by
the `assignWaveIndex` sequencer instruction.

Define the waveforms.

In [ ]:
x_array = np.linspace(-wfm_length//2, wfm_length//2, wfm_length)
sigma = wfm_length//8

# Define the waveforms as numpy arrays
waveform1 = np.array(
    np.exp(-np.power(x_array, 2.0) / (2 * np.power(sigma, 2.0))),
    dtype=float,
)
waveform2 = np.array(
    -x_array/sigma**2 * waveform1,
    dtype=float,
)

# Convert the numpy arrays to the native AWG waveform format
waveform_native12 = zhinst.utils.convert_awg_waveform(waveform1, waveform2)
waveform_native34 = zhinst.utils.convert_awg_waveform(waveform1, waveform2)

waveforms = [waveform_native12, waveform_native34]

Upload the native waveforms to the device.

In [ ]:
wfm_set = []
for wfm, awg_c in zip(waveforms, awg_cores):
    wfm_set.append([f"/{device_id:s}/awgs/{awg_c:d}/waveform/waves/{wfm_index:d}", wfm])

daq.set(wfm_set)

print("Waveforms upload successful")

### Enable the AWG 
This is the preferred method of using the AWG: run in single mode. Continuous waveform playback
is best achieved by using an infinite loop (e.g., `while (true)`) in the sequencer program.

Note that it is not necessary to enable all the AWG cores manually: by enabling one core, all the other are automatically enabled by the device. For this reason in this example only the first AWG core is enabled.

In [ ]:
daq.setInt(f"/{device_id:s}/awgs/{awg_cores[0]:d}/single", True)
daq.syncSetInt(f"/{device_id:s}/awgs/{awg_cores[0]:d}/enable", True)